In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import torch 

model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map=device)

Observing RTL-spec Pairs 

In [2]:
training_pairs = torch.load("dataset/rtl_spec_pairs.pth")
print("len(training_pairs):",len(training_pairs))
print("###################################################################################")

data_set = []
for i in range(3): 
    RTL = training_pairs[i][0]
    SPEC = training_pairs[i][1]

    print(RTL)
    print("...................................................................................")
    print(SPEC)
    print("###################################################################################")



len(training_pairs): 128
###################################################################################

module RefModule (
  output zero
);

  assign zero = 1'b0;

endmodule


...................................................................................
### Specification Description for `TopModule` Verilog Module

**Module Name:**  
TopModule

**Description:**  
The `TopModule` is a simple RTL (Register Transfer Level) Verilog module designed to consistently output a logic low (0) signal. The module has one bit wide input and output ports by default.

**Interface:**

- **Outputs:**
  - `zero`: A single-bit output port that should always be driven to a LOW state (logic level '0').

**Behavioral Description:**  
The primary function of this module is to maintain the `zero` output at logic low. There are no specific input ports or internal logic requirements, as the focus is solely on providing a constant zero output. This makes it suitable for applications where a known stati

Fine Tuning Specifications

In [3]:
# APPLYING PEFT TO MODEL 
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj', 'v_proj'])

model = get_peft_model(model, lora_config) 
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5484933737451648


In [4]:
# DEFINING LOSS FUNCTION 
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fn = nn.CrossEntropyLoss (reduction='none')
    cross_entropy_loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1)) 
    return cross_entropy_loss

In [5]:
# DEFINING OPTIMIZER
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

c:\Users\Andre\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
def generate_input_output_pair(prompt, target_responses):
    chat_templates = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False) 

    full_response_text = [
        (chat_template + " " + target_response + tokenizer.eos_token)
        for chat_template, target_response in zip(chat_templates, target_responses)
    ]
    input_ids_tokenized = tokenizer(full_response_text, return_tensors="pt", add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([" " + response + tokenizer.eos_token for response in target_responses],
            add_special_tokens=False, return_tensors="pt", padding="max_length", max_length=input_ids_tokenized.shape[1])["input_ids"]
        
    labels_tokenized_fixed = torch.where(labels_tokenized != tokenizer.pad_token_id, labels_tokenized, -100) 
    labels_tokenized_fixed[:, -1] = tokenizer.pad_token_id

    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1] 
    labels_tokenized_right_shifted  = labels_tokenized_fixed[:, 1:]
    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id

    return {"input_ids": input_ids_tokenized_left_shifted, "attention_mask": attention_mask, "labels": labels_tokenized_right_shifted}

Training Loop

In [7]:
from tqdm import tqdm

training_pairs = torch.load("dataset/rtl_spec_pairs.pth")
print("len(training_pairs):",len(training_pairs))
RTL = training_pairs[0][0]
SPEC = training_pairs[0][1]
user_intro = "Please generate a specification description for this rtl module:\n"

num_epochs = 20
iterations = len(training_pairs)

for epoch in range(num_epochs):
    total_loss = 0.0

    for i in tqdm(range(iterations), desc="Processing", unit="iteration"):
        RTL = training_pairs[i][0]
        SPEC = training_pairs[i][1]
        
        training_prompt = [
            { "role":"system", "content": "You are an AI assistant who answers questions."}, 
            { "role":"user", "content": user_intro + RTL}, 
            { "role":"assistant", "content":""}
        ]

        data = generate_input_output_pair(prompt=[training_prompt], target_responses=[SPEC])
        data["input_ids"] = data["input_ids"].to(device)
        data["labels"] = data["labels"].to(device)

        out = model(input_ids=data["input_ids"])
        loss = calculate_loss(out.logits, data["labels"]).mean()
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print("\tloss: ", loss.item())
    print(f" Epoch {epoch} total_loss:", total_loss)


len(training_pairs): 128


Processing:   0%|          | 0/128 [00:00<?, ?iteration/s]c:\Users\Andre\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\integrations\sdpa_attention.py:53: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Processing: 100%|██████████| 128/128 [02:31<00:00,  1.18s/iteration]


 Epoch 0 total_loss: 112.7109375


Processing:  74%|███████▍  | 95/128 [03:27<01:11,  2.18s/iteration]


KeyboardInterrupt: 

In [8]:
torch.save(model, "modeltrained.pth")

Observing Fine Tuned Output

In [9]:
training_pairs = torch.load("dataset/training_pairs.pth")
print("len(training_pairs):",len(training_pairs))

start = 128
iterations = 20 
user_intro = "Please generate a potential specification for this rtl module:\n"

for i in range(start,start+iterations):
    REFERENCE = training_pairs[i][1]

    training_prompt = [
            { "role":"system", "content": "You are an AI assistant who answers questions."}, 
            { "role":"user", "content": user_intro + REFERENCE}, 
            { "role":"assistant", "content":""}
        ]

    test_tokenized = tokenizer.apply_chat_template(training_prompt, continue_final_message=True, return_tensors="pt").to(device)
    test_out = model.base_model.generate(test_tokenized, max_new_tokens=500)
    print(tokenizer.batch_decode(test_out, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


len(training_pairs): 156


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 02 Mar 2025

You are an AI assistant who answers questions.user

Please generate a potential specification for this rtl module:

module RefModule (
  input clk,
  input aresetn,
  input x,
  output reg z
);

  parameter S=0, S1=1, S10=2;
  reg[1:0] state, next;

  always@(posedge clk, negedge aresetn)
    if (!aresetn)
      state <= S;
    else
      state <= next;

  always_comb begin
    case (state)
      S: next = x? S1 : S;
      S1: next = x? S1 : S10;
      S10: next = x? S1 : S;
      default: next = 'x;
    endcase
  end

  always_comb begin
    case (state)
      S: z = 0;
      S1: z = 0;
      S10: z = x;
      default: z = 'x;
    endcase
  end

endmoduleassistant

 ### Specification for Verilog Module: TopModule

#### Overview
The `TopModule` is designed to implement a 2-bit flip-flop with asynchronous reset. It operates on a single clock signal and uses asynchronous reset for its operation. The module has three i

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 02 Mar 2025

You are an AI assistant who answers questions.user

Please generate a potential specification for this rtl module:

module RefModule (
  input [3:0] a,
  input [3:0] b,
  input [3:0] c,
  input [3:0] d,
  input [3:0] e,
  output reg [3:0] q
);

  always @(*)
    case (c)
      0: q = b;
      1: q = e;
      2: q = a;
      3: q = d;
      default: q = 4'hf;
    endcase

endmoduleassistant

 # Specification for Verilog Module: TopModule

## Overview
The `TopModule` is a Verilog module designed to implement a 4-to-1 multiplexer with the specified functionality. It utilizes a single clock cycle to select between four input ports, producing one output port.

## Interface
- **Inputs:**
  - `a`: A 4-bit input.
  - `b`: A 4-bit input.
  - `c`: A 4-bit input.
  - `d`: A 4-bit input.
  - `e`: A 4-bit input.

- **Outputs:**
  - `q`: A 4-bit output.

## Specification
The module should implement a 4-to-1 multiplexer that selec

KeyboardInterrupt: 